# Environnement d'exécution

Allez dans "Fichier" puis "enregistrer une copie dans Drive" pour sauvegarder vos modifications personnelles.

Attention, avant de commencer à exécuter le code, il faut choisir un environnement T4 si ce n'est déjà fait (cliquer sur la petite flèche à droite de RAM et Disque en haut à droite).

# Installation des packages
Ça peut prendre 2 bonnes minutes, c'est normal.

In [ ]:
!pip install docling chonkie tiktoken sentence-transformers faiss-cpu transformers accelerate

# Import des packages

In [ ]:
from docling.document_converter import DocumentConverter # For text extraction
from chonkie import TokenChunker # For chunking
from sentence_transformers import SentenceTransformer  # For embeddings
import faiss  # For vector database
import numpy as np  # For numerical operations
from google.colab import files # to load a pdf file to test the system

# For answer generation with a small language model
# from transformers import pipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Chargement du PDF
Un bouton pour sélectionner le fichier à importer apparaît. Il suffit de faire la sélection.

In [ ]:
uploaded = files.upload()

# The uploaded file will be stored in a dictionary
file_name = list(uploaded.keys())[0]  # Get the name of the uploaded file

# Extraction du texte du PDF

In [ ]:
def extract_text_from_pdf(file_name,output_file="output.md"):
  converter = DocumentConverter()
  result = converter.convert(file_name)
  text=result.document.export_to_markdown()
  with open(output_file, "w", encoding="utf-8") as f:
    f.write(text)
  print(f"Markdown content saved to: {output_file}")
  return text

Pour la cellule ci-dessous, pensez à bien mettre le bon nom de fichier PDF. L'extraction du texte prend un peu de temps (1mn environ), c'est normal (un modèle d'OCR est chargé pour analyser le document et cela est chronophage). Une fois terminé, on peut regarder le resultat en ouvrant le fichier markdown dans les documents (panneau de gauche).

In [ ]:
### Extraction de text avec docling, attention
text = extract_text_from_pdf("DE_IA_UA_1_Rendu_Grégoire_Cesaro.pdf","output.md")

# Chunking
On découpe le texte extrait en petits morceaux (chunks) grâce au package Chonkie. Plusieurs techniques de chunking existent et il est bon d'en comparer plusieurs dans les cas d'usages en entreprise. Ici nous utilisons une méthode très simple qui découpe des chunks de taille chunk_size et avec un intersection de chunk_overlap entre chaque chunk.

In [ ]:
def process_text(text):
  # Initialize the chunker
  chunker = TokenChunker(chunk_size=90, chunk_overlap=30) # defaults to using GPT2 tokenizer
  # Chunk the text
  chunks = chunker(text)
  return chunks


In [ ]:
### tester le chunking
chunks= process_text(text)
# observer les chunks obtenus
for chunk in chunks:
    print(f"Chunk: {chunk.text}")
    print(f"Tokens: {chunk.token_count}")

In [ ]:
print(f"Total number of chunks: {len(chunks)}")

# Retrieval system
On utilise un modèle d'embeddings issu de la librairie transformers de huggingface pour transformer chaque chunk en un vecteur numérique. On va indexer et stocker ces vecteurs grâce à la librairie FAISS développée par Meta qui facilitera également la recherche de vecteurs similaires dans la suite.

In [ ]:
# Create retrieval system (FAISS index) with embeddings
def create_retriever(chunks):
    model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
    embeddings = model.encode(chunks, convert_to_tensor=True)
    embeddings_np = embeddings.cpu().numpy()

    dimension = embeddings_np.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings_np)
    return index, model

In [ ]:
# Create retrieval system
index, model = create_retriever(chunks)

# Identification des chunks les plus pertinents par rapport à la query

In [ ]:
# Retrieve relevant chunks based on query
def retrieve_info(index, model, query):
    # transformer la query en vecteur d'embedding
    query_embedding = model.encode(query, convert_to_tensor=True).cpu().numpy()
    # identifier les k chunks les plus pertinents pour cette query
    k = 3  # Number of nearest neighbors to retrieve
    scores, indices = index.search(query_embedding.reshape(1, -1), k)
    return scores, indices

# Example usage
# query = "votre demande en français ici"  # Replace with your French query
# scores, indices = retrieve_info(index, model, query)

In [ ]:
# Query: Question à poser au système
query= "Pour quelle raison on s'intéresse à l'utilisation de l'intelligence artificielle dans ce projet ?"

In [ ]:
# Retrieve relevant chunks
scores, indices = retrieve_info(index, model, query)

In [ ]:
# Regrouper les chunks les plus pertinents an un seul text qui constituera
# le context du prompt du modèle de language qui va générer une réponse à la question
retrieved_text = " ".join([chunks[idx].text for idx in indices[0]])
#answer = generate_answer(retrieved_text, query)
#print("\nAnswer:", answer)

In [ ]:
print(retrieved_text)

# Génération d'une réponse à la question (query)
Grâce à un small language model, en utilisant les chunks les plus pertinents par rapport à la question (contexte) et en formulant un prompt qui concatène ce contexte et la question

In [ ]:
# Chargement du modèle de language, ça peut prendre jusqu'à 1 minute environ

model_name = "ibm-granite/granite-3.1-2b-instruct"
# Chargement du tokenizer pour traiter le texte du prompt
tokenizer = AutoTokenizer.from_pretrained(model_name)
language_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="balanced",  # Using balanced CPU mapping.
    torch_dtype=torch.float16  # Use float16 if supported.
)
# mettre le modèle en mode "inférence"
language_model.eval()


In [ ]:
# prompt = query + context
chat = [
        { "role": "user", "content": f"Answer the following question: \n\n{query}\n\n based on the following information: \n\n{retrieved_text}" },
    ]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt").to("cuda:0")
#input_tokens = tokenizer(chat, return_tensors="pt").to("cpu") # si le modèle fonctionne avec du cpu
# generate output tokens
output = language_model.generate(**input_tokens,max_new_tokens=512)
# decode output tokens into text
output = tokenizer.batch_decode(output)
# print output
print(output)

# Améliorations
Ce TP est un système de RAG ultra basique et a été crée à des fins pédagogiques. De nombreuses améliorations et expérimentations peuvent être menées en partant de cette base.

*   Rendre le code plus propre. Par exemple, faire un code qui utilise toutes les fonctions créées ci dessus et qui récapitule tout (d'ailleurs il faudrait créer une fonction pour la dernière cellule de chat). En effet, ici, après chaque fonction, j'ajoute un test pour des raisons pédagogiques mais normalement le code devrait être structuré différemment.
*   Tester d'autres techniques de chunking. Explorer la documentation de Chonkie et tester d'autres techniques, tester différents paramètres... Pourquoi pas utiliser autre chose que Chonkie aussi.
*   Améliorer le retrieval: énormément de choses peuvent être envisagées comme utiliser un autre modèle d'embedding. Pourquoi pas tester du reranking ?
*   Améliorer le prompt, en tester plusieurs, ajouter une étape de reformulation de prompt.
*   Tester d'autres modèles de language pour la génération de la réponse. Faire varier les paramètres.





